In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data as pd_read
import functools 

SYM = "Symbol"
QTY = "Qty Available"
WEI = "Weight"
MEAN = "Mean"
STD = "Std"
VARI = "Variance"



In [ ]:
#read portfolio from actual zerodha sheet

PORTFOLIO_LOC = '../data/portfolio.xlsx' # replace the file in the data directory with you portfolio with name as portfolio.xlsx
portfolio_file = open(PORTFOLIO_LOC, 'rb')
portfolio = pd.read_excel(io = portfolio_file, usecols = "B,E", skiprows = 9, dtype =  {'b': np.str, 'e': np.int32} )

In [2]:
#ZERODHA sheet corrections 

#IDK WTF is AXISBANK6 in ZERODHA 
portfolio[SYM] = portfolio[SYM].replace(to_replace = "AXISBANK6", value = "AXISBANK")
#yahooo expects BS/NS suffix to symbol , assuming al stock in NSE
portfolio[SYM] += ".NS"

In [3]:
# make a weighted portfolio 
total_qty = portfolio[QTY].sum()
portfolio[WEI] = portfolio[QTY] / total_qty

In [4]:
# add mean std var for each stock to portfolio
stock_data = pd.DataFrame()
for stock in portfolio[SYM]:
    stock_data[stock]= pd_read.DataReader(stock, data_source='yahoo', start='2019-1-1')['Close']
stock_returns = np.log(stock_data/stock_data.shift(1))
portfolio = portfolio.merge(pd.DataFrame().join([stock_returns.mean().to_frame(MEAN),stock_returns.std().to_frame(STD),stock_returns.std().to_frame(VARI)],how = 'outer'),left_index = False, right_index = True, how = 'left', left_on = SYM) 
portfolio

,Symbol,Qty Available,Weight,Mean,Std,Variance
0,BHEL.NS,8,0.084211,-0.002048,0.034293,0.034293
1,HUDCO.NS,4,0.042105,-0.000659,0.029429,0.029429
2,LUPIN.NS,1,0.010526,0.000311,0.021205,0.021205
3,DABUR.NS,1,0.010526,0.000437,0.017174,0.017174
4,BIOCON.NS,4,0.042105,0.000542,0.022428,0.022428
5,NLCINDIA.NS,1,0.010526,-0.000751,0.024360,0.024360
6,LICHSGFIN.NS,1,0.010526,-0.001089,0.031348,0.031348
7,ITC.NS,4,0.042105,-0.001158,0.020607,0.020607
8,UPL.NS,1,0.010526,-0.000328,0.027490,0.027490
9,ICICIBANK.NS,1,0.010526,0.000231,0.027699,0.027699


In [5]:
# calculate covariance matrix 
cov_matrix = stock_returns.cov()

In [6]:
# portfolio risk
portfolio_risk = np.dot(portfolio[WEI].T, np.dot(cov_matrix, portfolio[WEI]))
portfolio_risk

0.00042397558599178195

In [7]:
#portfolio_volatility 
portfolio_volatility = portfolio_risk ** 0.5
portfolio_volatility

0.020590667448914374

In [8]:
#diversifiable risk  = portfolio risk - sum of  weigth **2 - variance for each stock 
diversifiabe_risk = portfolio_risk - (portfolio[WEI]**2 * portfolio[VARI]).sum()
diversifiabe_risk

-0.0029274674579269156